<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Papers/blob/main/P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

tf.__version__

'2.11.0'

In [24]:
vgg16 = tf.keras.applications.vgg16.VGG16(weights='imagenet')

In [25]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [27]:
vgg16_short = tf.keras.models.Model(vgg16.input, vgg16.layers[-5].output)

model = tf.keras.Sequential([
    vgg16_short, 
    tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding='same'),
    # tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding='same'),
    # tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2, padding='same'),
    # tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same'),
    # tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=4096),
    tf.keras.layers.Dense(units=7)
])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_11 (Functional)       (None, 7, 7, 512)         14714688  
                                                                 
 conv2d_33 (Conv2D)          (None, 7, 7, 1024)        4719616   
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 3, 3, 1024)       0         
 2D)                                                             
                                                                 
 conv2d_34 (Conv2D)          (None, 2, 2, 512)         4719104   
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 1, 1, 512)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 512)              

In [ ]:
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.vgg16 import preprocess_input


# Find the index of the first block3 layer
for index in range(len(vgg16.layers)):
    if 'block3' in vgg16.layers[index].name:
        break

# Add your own input
model_input = tf.keras.layers.Input(shape=(224,224,4), name='new_input')
x = tf.keras.layers.Conv2D(...)(model_input)
...

# Connect your last layer to the VGG16 model, starting at the "block3" layer
# Then, you need to connect every layer manually in a for-loop, freezing each layer along the way

for i in range(index, len(vgg16.layers)):
  vgg16.layers[i].trainable = False  

  x = vgg16.layers[i](x)

model_output = x
newModel = tf.keras.models.Model(model_input, model_output)

In [7]:
input2 = tf.keras.layers.Input(shape=(320, 320,))

# input2 = tf.keras.layers.Input(shape=(320, 320, 3))  #pick whatever the output of the previous model is
x = vgg16.get_layer("block5_conv1")(input2)
x = vgg16.get_layer("block5_conv2")(x)
x = vgg16.get_layer("block5_conv3")(x)
x = vgg16.get_layer("block5_pool")(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5) (x)
preds = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model_top = tf.keras.models.Model(input2, preds)

ValueError: ignored